In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sn
import cv2

In [ ]:
IMAGE_SIZE = 28
CHANNELS = 1
EPOCHS = 25
DIRECTORY = "E:\\PycharmProjects\\Sudoku_Solver"

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    f"{DIRECTORY}\\digits",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    color_mode='grayscale'
)

In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
def partition_dataset_tf(dataset, train_ratio=0.8, test_ratio=0.1, shuffle=True, shuffle_size=10000):
    assert (train_ratio + test_ratio) <= 1 # If smaller 1, a validation set is also created.
    
    dataset_size = len(dataset)
    
    if shuffle:
        dataset = dataset.shuffle(shuffle_size, seed=12)
        
    train_size = int(train_ratio * dataset_size)
    test_size = int(test_ratio * dataset_size)
    
    train_ds = dataset.take(train_size)
    test_ds = dataset.skip(train_size).take(test_size)
    val_ds = dataset.skip(train_size).skip(test_size)
    
    assert len(train_ds) + len(val_ds) + len(test_ds) == dataset_size
    
    return train_ds, val_ds, test_ds
        

In [ ]:
train_ds, val_ds, test_ds = partition_dataset_tf(dataset)

In [ ]:
print(f"train size:{len(train_ds)}, validation size:{len(val_ds)}, test size:{len(test_ds)}")

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resize_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1./255)
])

In [ ]:
plt.figure(figsize=(10, 10))
for image_batch, labels_batch in dataset.take(1):
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[labels_batch[i]])
        plt.axis("off")
    print(image_batch.shape)
    print(labels_batch.numpy())
    


In [ ]:
model = models.Sequential([
    resize_rescale,
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(100, activation='relu'),
    layers.Dense(10, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    verbose=1,
    epochs=EPOCHS
)

In [ ]:
model.evaluate(test_ds)

In [ ]:
predicted = model.predict(test_ds)
predicted_labels = [np.argmax(i) for i in predicted]
cm = tf.math.confusion_matrix(labels=test_ds, predictions=predicted_labels)

plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
for i in range(40):
    plt.matshow(test_ds[i])

In [ ]:
np.argmax(predicted[1])

In [ ]:
model.save("E:\PycharmProjects\Sudoku_Solver\model\Digit_Classifier")